In [2]:
import QuantLib as ql


In [5]:
today = ql.Date(7, ql.March, 2014)
ql.Settings.instance().evaluationDate = today

option = ql.EuropeanOption(ql.PlainVanillaPayoff(ql.Option.Call, 100.0),
                           ql.EuropeanExercise(ql.Date(7, ql.June, 2014)))

In [6]:
u = ql.SimpleQuote(100.0)
r = ql.SimpleQuote(0.01)
sigma = ql.SimpleQuote(0.20)

In [7]:
riskFreeCurve = ql.FlatForward(0, ql.TARGET(), ql.QuoteHandle(r), ql.Actual360())
volatility = ql.BlackConstantVol(0, ql.TARGET(), ql.QuoteHandle(sigma), ql.Actual360())

process = ql.BlackScholesProcess(ql.QuoteHandle(u),
                                 ql.YieldTermStructureHandle(riskFreeCurve),
                                 ql.BlackVolTermStructureHandle(volatility))


In [8]:
engine = ql.AnalyticEuropeanEngine(process)
option.setPricingEngine(engine)

In [9]:
print(option.NPV())
print(option.delta())
print(option.gamma())
print(option.vega())

4.155543462156206
0.5302223303784392
0.03934493301271913
20.109632428723106


In [77]:
help(QuantLib)

Help on module QuantLib.QuantLib in QuantLib:

NAME
    QuantLib.QuantLib

DESCRIPTION
    # This file was automatically generated by SWIG (http://www.swig.org).
    # Version 3.0.12
    #
    # Do not make changes to this file unless you know what you are doing--modify
    # the SWIG interface file instead.

CLASSES
    builtins.object
        ASX
        Array
        Average
        BackwardFlat
        BackwardFlatInterpolation
        Barrier
        BicubicSpline
        BilinearInterpolation
        BinomialDistribution
        Bisection
        BivariateCumulativeNormalDistribution
        BivariateCumulativeNormalDistributionDr78
        BivariateCumulativeNormalDistributionWe04DP
        BlackCalculator
        BlackDeltaCalculator
        BlackVolTermStructure
            BlackConstantVol
            BlackVarianceCurve
            BlackVarianceSurface
        BlackVolTermStructureHandle
            RelinkableBlackVolTermStructureHandle
        BondFunctions
        BoolVecto